In [1]:
import random,math,copy,timeit
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sklearn import svm
from sklearn import model_selection as ms

In [2]:
def random_search(n,dim):
    gens=[[0 for g in range(dim)] for _ in range(n)]
    for i,gen in enumerate(gens) :
        r=random.randint(1,dim)
        for _r in range(r):
            gen[_r]=1
        random.shuffle(gen)
    return gens



In [3]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= svm.LinearSVC().fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [4]:
def logsig(n): return 1 / (1 + math.exp(-n))
def sign(x): return 1 if x > 0 else (-1 if x!=0 else 0)

def BPSO(train_d,train_l,n=20,max_iter=200,w1=0.5,c1=0.5,c2=0.5,vmax=4):
    """
    input:{ 
            Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            dim: Number of feature, default=None
            prog: Do you want to use a progress bar?, default=False
            w1: move rate, default=0.5
            c1,c2: It's are two fixed variables, default=1,1
            vmax: Limit search range of vmax, default=4
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    dim=len(train_d[0])
    personal_best=float("-inf")
    global_best=float("-inf")
    gens=random_search(n,dim)
    vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]
    one_vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]
    zero_vel=[[random.random()-0.5 for d in range(dim)] for _n in range(n)]
    fit=[float("-inf") for i in range(n)]
    personal_best=dc(fit)
    xpersonal_best=dc(gens)
    global_best=max(fit)
    xglobal_best=gens[fit.index(max(fit))]
    gens_dict={tuple([0]*dim):float("-inf")}
    for it in range(max_iter):
        for i in range(n):
            if tuple(gens[i]) in gens_dict:
                score=gens_dict[tuple(gens[i])]
            else:
                score=evaluate(train_d,train_l,gens[i])
                gens_dict[tuple(gens[i])]=score
            fit[i]=score
            if fit[i]>personal_best[i]:#max
                personal_best[i]=dc(fit[i])
                xpersonal_best[i]=dc(gens[i])
        gg=max(fit)
        xgg=gens[fit.index(gg)]
        if global_best<gg:#max
            global_best=dc(gg)
            xglobal_best=dc(xgg)
        oneadd=[[0 for d in range(dim)] for i in range(n)]
        zeroadd=[[0 for d in range(dim)] for i in range(n)]
        c3=c1*random.random()
        dd3=c2*random.random()
        for i in range(n):
            for j in range(dim):
                if xpersonal_best[i][j]==0:
                    oneadd[i][j]=oneadd[i][j]-c3
                    zeroadd[i][j]=zeroadd[i][j]+c3
                else:
                    oneadd[i][j]=oneadd[i][j]+c3
                    zeroadd[i][j]=zeroadd[i][j]-c3
                if xglobal_best[j]==0:
                    oneadd[i][j]=oneadd[i][j]-dd3
                    zeroadd[i][j]=zeroadd[i][j]+dd3
                else:
                    oneadd[i][j]=oneadd[i][j]+dd3
                    zeroadd[i][j]=zeroadd[i][j]-dd3
        one_vel=[[w1*_v+_a for _v,_a in zip(ov,oa)] for ov,oa in zip(one_vel,oneadd)]
        zero_vel=[[w1*_v+_a for _v,_a in zip(ov,oa)] for ov,oa in zip(zero_vel,zeroadd)]
        for i in range(n):
            for j in range(dim):
                if abs(vel[i][j]) > vmax:
                    zero_vel[i][j]=vmax*sign(zero_vel[i][j])
                    one_vel[i][j]=vmax*sign(one_vel[i][j])
        for i in range(n):
            for j in range(dim):
                if gens[i][j]==1:
                    vel[i][j]=zero_vel[i][j]
                else:
                    vel[i][j]=one_vel[i][j]
        veln=[[logsig(s[_s]) for _s in range(len(s))] for s in vel]
        temp=[[random.random() for d in range(dim)] for _n in range(n)]
        for i in range(n):
            for j in range(dim):
                if temp[i][j]<veln[i][j]:
                    gens[i][j]= 0 if gens[i][j] ==1 else 1
                else:
                    pass
    return xglobal_best,xglobal_best.count(1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import SVC
import numpy as np

def test_score(gen, tr_x, tr_y, te_x, te_y):
    mask = np.array(gen) == 1
    al_data = np.array(tr_x[:, mask])
    al_test_data = np.array(te_x[:, mask])

    accuracies, precisions, recalls, f1s = [], [], [], []
    for _ in range(5):
        model = SVC(kernel='rbf', C=1.0, gamma='scale') s
        model.fit(al_data, tr_y)
        preds = model.predict(al_test_data)

        accuracies.append(accuracy_score(te_y, preds))
        precisions.append(precision_score(te_y, preds, zero_division=0))
        recalls.append(recall_score(te_y, preds, zero_division=0))
        f1s.append(f1_score(te_y, preds, zero_division=0))

    return (
        np.mean(accuracies),
        np.mean(precisions),
        np.mean(recalls),
        np.mean(f1s)
    )

In [12]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

with open("/Users/ximenacanton/Documents/TEC/5 SEM/bio inspirado/reto/Bio-inspired-algorithms-for-diagnosis-of-breast-cancer/wdbc.data.csv") as f:
    lines = f.read().splitlines()

# id (columna 0), diagnosis (columna 1), resto son features numéricas
x = np.array([[float(d) for d in line.split(',')[2:]] for line in lines])  # desde la 3ra columna
y = np.array([line.split(',')[1] for line in lines])  # diagnosis ('M' o 'B')

lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)

train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)


print("Shape X:", x.shape)

Shape X: (569, 30)


In [13]:

k = [1 for r in range(len(x[0]))]
print(test_score(k, train_d, train_l, test_d, test_l))

fattr = 0
# per-metric accumulators
ftest_acc = 0.0
ftest_prec = 0.0
ftest_rec = 0.0
ftest_f1 = 0.0

flist = [0 for _ in range(len(x[0]))]
final_list = [0 for _ in range(len(x[0]))]

start = timeit.default_timer()

for i in range(20):
    g, l = BPSO(train_d,train_l,n=20,max_iter=200,w1=0.5,c1=0.5,c2=0.5,vmax=4)
    
    # Asegurar que 'l' sea numérico
    if isinstance(l, list):
        l = len(l)
    elif not isinstance(l, (int, float)):
        l = float(l)

    fattr += l

    # unpack metrics
    acc, prec, rec, f1 = test_score(g, train_d, train_l, test_d, test_l)
    ftest_acc += acc
    ftest_prec += prec
    ftest_rec += rec
    ftest_f1 += f1
    
    for j in range(len(flist)):
        if g[j] == 1:
            flist[j] += 1

    # print per-iteration metrics (Acc and F1 shown)
    print(f"{i + 1:2d}  {''.join(map(str, g))}  {l:.4f}  Acc:{acc:.4f}  F1:{f1:.4f}")

# Promedios
fattr /= 20
ftest_acc /= 20
ftest_prec /= 20
ftest_rec /= 20
ftest_f1 /= 20

end = timeit.default_timer()
time = end - start

# Asegurar que fattr esté dentro de rango y sea entero
final_count = int(round(fattr))
final_count = max(0, min(final_count, len(flist)))

final = np.argsort(flist)[::-1][:final_count]

for idx in final:
    final_list[idx] = 1

print(f"Final:  {''.join(map(str, final_list))}   {fattr:.4f}   Acc:{ftest_acc:.6f}  F1:{ftest_f1:.6f}    {time:.4f}")

(np.float64(0.8951048951048952), np.float64(0.9782608695652174), np.float64(0.7627118644067796), np.float64(0.8571428571428571))
 1  001011011100001011101110111111  19.0000  Acc:0.9371  F1:0.9189
 2  001001011100000110001110111110  15.0000  Acc:0.9371  F1:0.9189
 3  001011110100001101011100101111  17.0000  Acc:0.9161  F1:0.8889
 4  000010010101111001000110100110  13.0000  Acc:0.9301  F1:0.9107
 5  001011010100000110011110111110  16.0000  Acc:0.9371  F1:0.9189
 6  001000011000001001111110111110  15.0000  Acc:0.9371  F1:0.9189
 7  001000010000001111001110101100  12.0000  Acc:0.9371  F1:0.9189
 8  001000010100001011001100010101  11.0000  Acc:0.9231  F1:0.8991
 9  001010011100001100001110001111  14.0000  Acc:0.9371  F1:0.9189
10  011000010100000000101100101110  11.0000  Acc:0.9231  F1:0.9027
11  110100110100000000111010110100  13.0000  Acc:0.8811  F1:0.8350
12  001010011000000110001110001100  11.0000  Acc:0.9371  F1:0.9189
13  000011011011110011110110100101  17.0000  Acc:0.9231  F1:0.9009
